# FRQ 3
> Relationship of FRQ 3 to a Project Based learning exercise.
- title: FRQ3 - Array/ArrayList
- toc: true
- badges: false
- image: /images/frqs.png
- categories: [1.B]
- tags: [api]
- type: ap
- week: 13

## Calculator
> Helping a computer interpret a Mathematical expression is a derivation of FRQ3 that involves using ArrayLists, Stack, and Map.  These are key data structures that exist in Java.  To obtain a deeper understanding of ArrayList, or its List interface, it is to use it in many applications.

### Operators and Precedence
> To support mathematical expressions you need to define Unique List of operators that are supported by your Calculator.

In [ ]:
// Helper definition to define operators, lookup in MAP are fast and easy O(1) versus ArrayList O(n)
private final Map<String, Integer> OPERATORS = new HashMap<>();
{
    // Map<"token", precedence>
    OPERATORS.put("*", 3);
    OPERATORS.put("/", 3);
    OPERATORS.put("%", 3);
    OPERATORS.put("+", 4);
    OPERATORS.put("-", 4);
}

### Organizing an Expression
> To support terms in mathematical expression you need to define symbols (other than operators) that help delineate the elements of an expression.  Ultimately, the String expression will be broken in distinct elements in a List, we will call each element a Token.

In [ ]:

// Helper definition for supported separators
private final Map<String, Integer> SEPARATORS = new HashMap<>();
{
    // Map<"separator", not_used>
    SEPARATORS.put(" ", 0);
    SEPARATORS.put("(", 0);
    SEPARATORS.put(")", 0);
}

### Expression Evaluation
To detect operators and separators in the String expression, requires test functions to detect the tokens. These methods will assist in establishing Control Flow in evaluating the expression.

In [ ]:
// Test if token is an operator
private boolean isOperator(String token) {
    // find the token in the hash map
    return OPERATORS.containsKey(token);
}

// Test if token is an separator
private boolean isSeparator(String token) {
    // find the token in the hash map
    return SEPARATORS.containsKey(token);
}

// Compare precedence of operators.
private Boolean isPrecedent(String token1, String token2) {
    // token 1 is precedent if it is greater than token 2
    return (OPERATORS.get(token1) - OPERATORS.get(token2) >= 0) ;
}

## Calculator Theory
> Mathematical expression written by humans are in the form of a String Expression.  Since this expression is simply a string to the computer...  an algorithm is required to reform the equation for the computer to ensure it interprets expression according to the rules of mathematics. 
- In mathematics, an expression or mathematical expression is a finite combination of symbols that is well-formed (ie "3 + 4").
- In computers, a string expression is hard to calculate.  Thus, the expression needs to be restructured according to rules of Mathematics that a computer can calculate.  For instance, the order of precedence rules, aka PEMDAS (parenthesis, exponents, multiplication, division, addition, subtraction) need to be factored into order of computation. To support these rules, in computer math we often convert a String expression into Reverse Polish Notation (RPN).  This converts a simple string like "3 + 4" to become ["3", "4", "+"]) using the Shunting-yard algorithm.

> After thinking about basic computing requirements of an expression and need for an RPN algorithm, step back and think of flow of control to go from defining terms/tokens, to RPN, and ultimately calculate the final result.  This flow of control is illustrated in the Calculator instance variables and constructor as shown.  Also, listed is a toString method to output results.

In [ ]:
// Key instance variables
private final String expression;
private ArrayList<String> tokens;
private ArrayList<String> reverse_polish;
private Double result;


// Create a 1 argument constructor providing a mathematical expression
    public Calculator(String expression) {
        // original input
        this.expression = expression;

        // parse expression into terms
        this.termTokenizer();

        // place terms into reverse polish notation
        this.tokensToReversePolishNotation();

        // calculate reverse polish notation
        this.rpnToResult();
    }

    // Print the expression, terms, and result
    public String toString() {
        return ("Original expression: " + this.expression + "\n" +
                "Tokenized expression: " + this.tokens.toString() + "\n" +
                "Reverse Polish Notation: " +this.reverse_polish.toString() + "\n" +
                "Final result: " + String.format("%.2f", this.result));
    }

### Term Tokenizer
> A Term tokenizer is used to change the String expression into a series of tokens that constitute distinct elements of a Mathematical expression.  The result is retained in the ArrayList "tokens" instance variable.

In [ ]:
// Term Tokenizer takes original expression and converts it to ArrayList of tokens
private void termTokenizer() {
    // contains final list of tokens
    this.tokens = new ArrayList<>();

    int start = 0;  // term split starting index
    StringBuilder multiCharTerm = new StringBuilder();    // term holder
    for (int i = 0; i < this.expression.length(); i++) {
        Character c = this.expression.charAt(i);
        if ( isOperator(c.toString() ) || isSeperator(c.toString())  ) {
            // 1st check for working term and add if it exists
            if (multiCharTerm.length() > 0) {
                tokens.add(this.expression.substring(start, i));
            }
            // Add operator or parenthesis term to list
            if (c != ' ') {
                tokens.add(c.toString());
            }
            // Get ready for next term
            start = i + 1;
            multiCharTerm = new StringBuilder();
        } else {
            // multi character terms: numbers, functions, perhaps non-supported elements
            // Add next character to working term
            multiCharTerm.append(c);
        }

    }
    // Add last term
    if (multiCharTerm.length() > 0) {
        tokens.add(this.expression.substring(start));
    }
}

### Tokens to RPN
> Before calculation the tokens need to be turned into RPN.  This results in the operator being followed by its operands.

In [ ]:
// Takes tokens and converts to Reverse Polish Notation (RPN).
private void tokensToReversePolishNotation () {
    // contains final list of tokens in RPN
    List<String> reverse_polish = new ArrayList<>();

    // stack is used to reorder for appropriate grouping and precedence
    Stack tokenStack = new Stack();
    for (String token : tokens) {
        switch (token) {
            // If left bracket push token on to stack
            case "(":
                tokenStack.push(token);
                break;
            case ")":
                while (tokenStack.peek() != null && !tokenStack.peek().equals("("))
                {
                    reverse_polish.add( (String)tokenStack.pop() );
                }
                tokenStack.pop();
                break;
            case "+":
            case "-":
            case "*":
            case "/":
            case "%":
                // While stack
                // not empty AND stack top element
                // and is an operator
                while (tokenStack.peek() != null && isOperator((String) tokenStack.peek()))
                {
                    if ( isPrecedent(token, (String) tokenStack.peek() )) {
                        reverse_polish.add((String)tokenStack.pop());
                        continue;
                    }
                    break;
                }
                // Push the new operator on the stack
                tokenStack.push(token);
                break;
            default:    // Default should be a number, there could be test here
                this.reverse_polish.add(token);
        }
    }
    // Empty remaining tokens
    while (tokenStack.peek() != null) {
        reverse_polish.add((String)tokenStack.pop());
    }

}

### RPN to Result
> Below is outline/pseudo code to produce a result.

In [ ]:
// Takes RPN and produces a final result
private void rpnToResult()
{
    // Stack used to hold calculation while process RPN
    Stack calculation = new Stack();

    // for loop to process RPN
    {
        // If the token is a number
        {
            // Push number to stack
        }
        // else
        {
            // Pop the two top entries

            // Based off of Token operator calculate result

            // Push result back onto the stack
        }
    }
    // Pop final result and set as final result for expression
}

## Driver / Tester
> A calculator should always have a driver for testing.  In a driver, you will need to consider multiple conditions, for instance changes with precedence...

In [ ]:
Calculator simpleMath = new Calculator("100 + 200  * 3");
System.out.println("Simple Math\n" + simpleMath);

Calculator parenthesisMath = new Calculator("(100 + 200)  * 3");
System.out.println("Parenthesis Math\n" + parenthesisMath);

Calculator allMath = new Calculator("200 % 300 + 5 + 300 / 200 + 1 * 100");
System.out.println("All Math\n" + allMath);

Calculator allMath2 = new Calculator("200 % (300 + 5 + 300) / 200 + 1 * 100");
System.out.println("All Math2\n" + allMath2);

### Output Examples
> Finally, here is sample output for the calculator

```text
Simple Math
Original expression: 100 + 200  * 3
Tokenized expression: [100, +, 200, *, 3]
Reverse Polish Notation: [100, 200, 3, *, +]
Final result: 700.00

Parenthesis Math
Original expression: (100 + 200)  * 3
Tokenized expression: [(, 100, +, 200, ), *, 3]
Reverse Polish Notation: [100, 200, +, 3, *]
Final result: 900.00

Fraction Math
Original expression: 100.2 - 99.3
Tokenized expression: [100.2, -, 99.3]
Reverse Polish Notation: [100.2, 99.3, -]
Final result: 0.90

Modulo Math
Original expression: 300 % 200
Tokenized expression: [300, %, 200]
Reverse Polish Notation: [300, 200, %]
Final result: 100.00

Division Math
Original expression: 300/200
Tokenized expression: [300, /, 200]
Reverse Polish Notation: [300, 200, /]
Final result: 1.50

Multiplication Math
Original expression: 300 * 200
Tokenized expression: [300, *, 200]
Reverse Polish Notation: [300, 200, *]
Final result: 60000.00

All Math
Original expression: 200 % 300 + 5 + 300 / 200 + 1 * 100
Tokenized expression: [200, %, 300, +, 5, +, 300, /, 200, +, 1, *, 100]
Reverse Polish Notation: [200, 300, %, 5, +, 300, 200, /, +, 1, 100, *, +]
Final result: 306.50

All Math2
Original expression: 200 % (300 + 5 + 300) / 200 + 1 * 100
Tokenized expression: [200, %, (, 300, +, 5, +, 300, ), /, 200, +, 1, *, 100]
Reverse Polish Notation: [200, 300, 5, +, 300, +, %, 200, /, 1, 100, *, +]
Final result: 101.00

All Math3
Original expression: 200%(300+5+300)/200+1*100
Tokenized expression: [200, %, (, 300, +, 5, +, 300, ), /, 200, +, 1, *, 100]
Reverse Polish Notation: [200, 300, 5, +, 300, +, %, 200, /, 1, 100, *, +]
Final result: 101.00
```

## Hacks
> Build a calculator to process expressions and ultimately change RPN to a calculation.
1. Finish Calculator
2. Build in Power of operator ^: 2 ^ 1 = 2, 2 ^ 2 = 4, 2 ^ 3 = 8

> Beyond short term, including PBL ideas.
3. Build variable assignment and evaluation into your expressions (a = 2; a + 1).
4. Try adding single argument function SQRT.
5. Consider design to capture expression in FE and pass expression to BE.  Consider how you might maintain variables on FE.  